# Team 4_4 Join from Raw datasets

## Join Description

## Dataset Feature Guide

Below is a concise field guide for your final dataset. For each feature we list: **Type**, **Source**, **When available**, **Leakage?**, and **Description / usage notes**.

---

### Keys & Provenance (do not drop)
| Feature | Type | Source | When available | Leakage? | Description / usage notes |
|---|---|---|---|---|---|
| `flight_id` | string | Derived | Always | No | Stable key: `FL_DATE|OP_UNIQUE_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST`. Use for joins/auditing. |
| `prediction_utc` | timestamp (UTC) | Derived | **T–2h** | No | Decision time in UTC used to pick “as-of” weather. Built from local schedule + airport timezone. |
| `origin_obs_utc` | timestamp (UTC) | Weather | **T–2h** | No | Timestamp of the weather observation actually used for origin (≤ `prediction_utc`). |
| `asof_minutes` | int | Derived | **T–2h** | No | Freshness of weather in minutes: `prediction_utc − origin_obs_utc` (0 = same hour; larger = older). |
| `origin_station_id` | string | Weather | **T–2h** | No | NOAA station chosen for origin (nearest then latest within 6h lookback). |
| `dest_station_id` | string | Weather | Pre-computed | No | Rank-1 nearest station to destination (location helpers only; no dest weather attached). |

---

### Flight & Schedule (model inputs)
| Feature | Type | Source | When available | Leakage? | Description / usage notes |
|---|---|---|---|---|---|
| `FL_DATE` | date | Flights | Always | No | Flight local calendar date. Combine with CRS times for timestamps. |
| `YEAR` / `QUARTER` / `MONTH` / `DAY_OF_MONTH` / `DAY_OF_WEEK` | int | Flights | Always | No | Calendar fields for seasonality/day-of-week effects. |
| `OP_UNIQUE_CARRIER` | string | Flights | Always | No | Carrier code (e.g., AA, DL). Categorical; encode appropriately. |
| `OP_CARRIER` | string | Flights | Always | No | Often duplicative of `OP_UNIQUE_CARRIER`; keep one to avoid redundancy. |
| `OP_CARRIER_FL_NUM` | int | Flights | Always | No | Flight number. High-cardinality; consider hashing or drop if not needed. |
| `TAIL_NUM` | string | Flights | Often | No | Aircraft tail (optional; sparse/reused). Use with care to avoid overfitting. |
| `CRS_DEP_TIME` | int (HHMM) | Flights | Always | No | Scheduled departure local time (e.g., 755 ⇒ 07:55). Derive hour buckets if useful. |
| `CRS_ARR_TIME` | int (HHMM) | Flights | Always | No | Scheduled arrival local time. |
| `CRS_ELAPSED_TIME` | int (min) | Flights | Always | No | Planned duration in minutes; helps normalize route length. |
| `ORIGIN` / `DEST` | string (IATA) | Flights | Always | No | Three-letter airport codes. Join keys to airport dim and station bridge. |
| `ORIGIN_AIRPORT_ID` / `DEST_AIRPORT_ID` | int | Flights | Always | No | Numeric airport identifiers (BTS). |
| `ORIGIN_CITY_NAME` / `DEST_CITY_NAME` | string | Flights | Always | No | Human-readable city names (may be redundant). |
| `ORIGIN_STATE_ABR` / `DEST_STATE_ABR` | string | Flights | Always | No | Two-letter state codes. |
| `DISTANCE` | double (miles) | Flights | Always | No | Great-circle distance; often inversely related to dep-delay rate. |
| `DISTANCE_GROUP` | int | Flights | Always | No | Distance bucket (BTS bins). Categorical alternative to continuous miles. |

---

### Origin Weather (as-of T–2h; model inputs)
> NOAA ISD fields often arrive as strings with units/qualifiers; parse to numeric and handle missing (“M”) as needed.

| Feature | Type | Source | When available | Leakage? | Description / usage notes |
|---|---|---|---|---|---|
| `HourlyDryBulbTemperature` | string → numeric | Weather | T–2h | No | Ambient air temperature. Parse numeric; watch units/qualifiers. |
| `HourlyDewPointTemperature` | string → numeric | Weather | T–2h | No | Dew point; with RH helps assess fog/icing risk. |
| `HourlyWetBulbTemperature` | string → numeric | Weather | T–2h | No | Wet-bulb temp; proxy for heat/humidity effects. |
| `HourlyPrecipitation` | string → numeric (in) | Weather | T–2h | No | Hourly precip. Many zeros; spikes predictive. |
| `HourlyWindSpeed` | string → numeric | Weather | T–2h | No | Sustained wind speed. Consider last-6h max as extra feature. |
| `HourlyWindDirection` | string → numeric (deg) | Weather | T–2h | No | 0–360°; consider sin/cos transform for cyclicity. |
| `HourlyWindGustSpeed` | string → numeric | Weather | T–2h | No | Gusts; sparse but predictive when present. |
| `HourlyVisibility` | string → numeric (mi) | Weather | T–2h | No | Surface visibility; lower ⇒ more delay risk. |
| `HourlyRelativeHumidity` | string → numeric (%) | Weather | T–2h | No | High RH + low visibility often impactful. |
| `HourlyStationPressure` | string → numeric (mb) | Weather | T–2h | No | Station pressure; consider deltas if available. |
| `HourlySeaLevelPressure` | string → numeric (mb) | Weather | T–2h | No | SLP; quality varies by station. |
| `HourlyAltimeterSetting` | string → numeric (inHg) | Weather | T–2h | No | Altimeter setting. |
| `HourlySkyConditions` | string (coded) | Weather | T–2h | No | Tokenize layers (FEW/BKN/OVC + heights). Multi-hot categorical works well. |
| `HourlyPresentWeatherType` | string (coded) | Weather | T–2h | No | Phenomena codes (RA, SN, FG, TS…). Parse into flags per phenomenon. |

---

### Location Helpers (model inputs; not labels)
| Feature | Type | Source | When available | Leakage? | Description / usage notes |
|---|---|---|---|---|---|
| `origin_station_lat` / `origin_station_lon` | double | Weather | T–2h | No | Coordinates of the **selected** origin station used for as-of weather. |
| `origin_airport_lat` / `origin_airport_lon` | double | Airports | Always | No | Airport centroid coordinates (from airport dim). |
| `dest_station_lat` / `dest_station_lon` | double | Weather | Pre-computed | No | Coordinates of nearest dest station (rank-1), for geography features only. |
| `dest_airport_lat` / `dest_airport_lon` | double | Airports | Always | No | Destination airport coordinates. |
| `origin_station_dis` | double (km) | Derived | T–2h | No | Haversine distance airport ↔ **chosen** origin station. Smaller is better. |
| `dest_station_dis` | double (km) | Derived | Pre-computed | No | Haversine distance destination airport ↔ rank-1 station. |
| `origin_type` | string | Airports (airport-codes CSV) | Always | No | Airport facility type for ORIGIN (e.g., `large_airport`, `medium_airport`, `small_airport`, `heliport`, `seaplane_base`). Useful categorical capturing operational complexity; one-hot or target-encode. |
| `dest_type` | string | Airports (airport-codes CSV) | Always | No | Airport facility type for DEST; same coding as `origin_type`. |


---

### Targets & Evaluation (keep; **never** feed to model)
| Feature | Type | Source | When available | Leakage? | Description / usage notes |
|---|---|---|---|---|---|
| `DEP_DEL15` | int (0/1) | Flights | After departure | **Yes** | Primary label: 1 if departure delay ≥ 15 min. Target only. |
| `DEP_DELAY` | double (min) | Flights | After departure | **Yes** | Continuous dep delay. Useful for calibration/cost curves. |
| `ARR_DEL15` | int (0/1) | Flights | After arrival | **Yes** | Optional secondary outcome for reports (not part of dep-delay model). |
| `ARR_DELAY` | double (min) | Flights | After arrival | **Yes** | Arrival delay; EDA only. |

---

### Post-Flight Diagnostics (EDA/debug; **never** feed to model)
| Feature | Type | Source | When available | Leakage? | Description / usage notes |
|---|---|---|---|---|---|
| `CARRIER_DELAY` | double (min) | Flights | Post-flight | **Yes** | Minutes attributed to carrier. Diagnostic only. |
| `WEATHER_DELAY` | double (min) | Flights | Post-flight | **Yes** | Minutes due to weather (BTS coding). |
| `NAS_DELAY` | double (min) | Flights | Post-flight | **Yes** | National Airspace System delay minutes. |
| `SECURITY_DELAY` | double (min) | Flights | Post-flight | **Yes** | Minutes due to security. |
| `LATE_AIRCRAFT_DELAY` | double (min) | Flights | Post-flight | **Yes** | Minutes due to late inbound aircraft. |
| `DEP_TIME` / `ARR_TIME` | int (HHMM) | Flights | Post-flight | **Yes** | Actual departure/arrival local time. |
| `TAXI_OUT` / `TAXI_IN` | int (min) | Flights | Post-flight | **Yes** | Actual taxi times. |
| `WHEELS_OFF` / `WHEELS_ON` | int (HHMM) | Flights | Post-flight | **Yes** | Actual off/on times. |
| `ACTUAL_ELAPSED_TIME` | int (min) | Flights | Post-flight | **Yes** | Actual duration. |
| `AIR_TIME` | int (min) | Flights | Post-flight | **Yes** | In-air time. |

---

### Curation Flags (keep until filtered)
| Feature | Type | Source | When available | Leakage? | Description / usage notes |
|---|---|---|---|---|---|
| `CANCELLED` | int (0/1) | Flights | Always | — | Filter to `0` for dep-delay modeling. |
| `CANCELLATION_CODE` | string | Flights | If cancelled | — | Reason (A=Carrier, B=Weather, C=NAS, D=Security). For analysis only. |
| `DIVERTED` | int (0/1) | Flights | After arrival | — | Filter to `0` for dep-delay modeling. |

---

### Practical Modeling Notes
- **Time handling:** Weather joins are UTC and **as-of** `prediction_utc` (no future look-ahead).  
- **Parsing weather:** Convert NOAA strings to numeric; add binary flags (e.g., precip>0, gust present).  
- **Cyclical times:** Derive `CRS_DEP_HOUR` and encode with sin/cos.  
- **High-cardinality:** `TAIL_NUM` and `OP_CARRIER_FL_NUM` can overfit; drop/hash/regularize.  
- **Collinearity:** `OP_UNIQUE_CARRIER` vs `OP_CARRIER` are often duplicates—keep one.  
- **Leakage guard:** Never include any “actual” time or delay field in training features.


## Load parquet file

In [0]:
# Read 5Y Joined dataset from raw files 
out_path = "dbfs:/student-groups/Group_4_4/JOINED_5Y_2015_2019.parquet"
df_joined_5Y_2015 = spark.read.parquet(out_path)
print(f"Our Join row count: {df_joined_5Y_2015.count()}")

# Read 5Y Flights dataset from raw files
df_flights = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/")
print(f"Flights raw dataset row count: {df_flights.count()}")


Our Join row count: 31746841
Flights raw dataset row count: 74177433


In [0]:
display(df_joined_5Y_2015.orderBy(df_joined_5Y_2015.columns[0], ascending=True).limit(5))
display(df_joined_5Y_2015.orderBy(df_joined_5Y_2015.columns[0], ascending=False).limit(5))

flight_id,prediction_utc,origin_obs_utc,asof_minutes,origin_station_id,dest_station_id,FL_DATE,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER,OP_CARRIER_FL_NUM,TAIL_NUM,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,ORIGIN,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_STATE_ABR,DISTANCE,DISTANCE_GROUP,HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyWetBulbTemperature,HourlyPrecipitation,HourlyWindSpeed,HourlyWindDirection,HourlyWindGustSpeed,HourlyVisibility,HourlyRelativeHumidity,HourlyStationPressure,HourlySeaLevelPressure,HourlyAltimeterSetting,HourlySkyConditions,HourlyPresentWeatherType,origin_station_lat,origin_station_lon,origin_airport_lat,origin_airport_lon,dest_station_lat,dest_station_lon,dest_airport_lat,dest_airport_lon,origin_station_dis,dest_station_dis,origin_type,dest_type,DEP_DEL15,DEP_DELAY,ARR_DEL15,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DEP_TIME,ARR_TIME,TAXI_OUT,TAXI_IN,WHEELS_OFF,WHEELS_ON,ACTUAL_ELAPSED_TIME,AIR_TIME,CANCELLED,CANCELLATION_CODE,DIVERTED
2015-01-01|AA|1001|DFW|TUS,2015-01-01T22:25:00Z,2015-01-01T21:53:00Z,32,72259003927,72274023160,2015-01-01,2015,1,1,1,4,AA,AA,1001,N4WTAA,1825,1945,140.0,DFW,11298,"Dallas/Fort Worth, TX",TX,TUS,15376,"Tucson, AZ",AZ,813.0,4,36,34,35,T,7,360,null,4.00,93,29.58,30.24,30.23,SCT:04 6 OVC:08 19,-DZ:01 BR:1 |DZ |,32.898,-97.019,32.89651945,-97.0465220537124,32.131,-110.955,32.120244549999995,-110.93687678656917,2.57485082431794,2.0839776075153473,large_airport,medium_airport,1.0,76.0,1.0,77.0,25.0,0.0,1.0,0.0,51.0,1941,2102,14.0,4.0,1955,2058,141.0,123.0,0.0,null,0.0
2015-01-01|AA|1002|DFW|IAH,2015-01-01T14:30:00Z,2015-01-01T14:28:00Z,2,72259003927,72243012960,2015-01-01,2015,1,1,1,4,AA,AA,1002,N475AA,1030,1135,65.0,DFW,11298,"Dallas/Fort Worth, TX",TX,IAH,12266,"Houston, TX",TX,224.0,1,35,33,34,T,7,010,null,1.50,93,29.57,null,30.22,OVC:08 4,-DZ:01 |DZ |,32.898,-97.019,32.89651945,-97.0465220537124,29.98,-95.36,29.9841416,-95.33298595614491,2.57485082431794,2.6423037301579435,large_airport,large_airport,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,B,0.0
2015-01-01|AA|1002|IAH|DFW,2015-01-01T16:25:00Z,2015-01-01T15:53:00Z,32,72243012960,72259003927,2015-01-01,2015,1,1,1,4,AA,AA,1002,N559AA,1225,1340,75.0,IAH,12266,"Houston, TX",TX,DFW,11298,"Dallas/Fort Worth, TX",TX,224.0,1,45,43,44,0.01,8,030,null,2.00,93,30.01,30.12,30.12,OVC:08 6,-RA:02 BR:1 |RA |,29.98,-95.36,29.9841416,-95.33298595614491,32.898,-97.019,32.89651945,-97.0465220537124,2.6423037301579435,2.57485082431794,large_airport,large_airport,0.0,-1.0,0.0,-11.0,null,null,null,null,null,1224,1329,11.0,17.0,1235,1312,65.0,37.0,0.0,null,0.0
2015-01-01|AA|1003|MIA|PHL,2015-01-01T12:40:00Z,2015-01-01T11:53:00Z,47,72202012839,72408013739,2015-01-01,2015,1,1,1,4,AA,AA,1003,N3BXAA,940,1233,173.0,MIA,13303,"Miami, FL",FL,PHL,14100,"Philadelphia, PA",PA,1013.0,5,79,67,71,0.00,7,110,null,10.00,67,30.15,30.18,30.18,SCT:04 25 BKN:07 55 OVC:08 250,null,25.788,-80.317,25.7949789,-80.28672341095405,39.873,-75.227,39.875018,-75.23521278969667,3.1289947807252796,0.7359028498396194,large_airport,large_airport,0.0,1.0,0.0,-14.0,null,null,null,null,null,941,1219,14.0,4.0,955,1215,158.0,140.0,0.0,null,0.0
2015-01-01|AA|1003|PHL|MIA,2015-01-01T16:25:00Z,2015-01-01T15:54:00Z,31,72408013739,72202012839,2015-01-01,2015,1,1,1,4,AA,AA,1003,N3BXAA,1325,1618,173.0,PHL,14100,"Philadelphia, PA",PA,MIA,13303,"Miami, FL",FL,1013.0,5,40,15,32,0.00,14,230,null,10.00,36,30.02,30.04,30.05,SCT:04 250,null,39.873,-75.227,39.875018,-75.23521278969667,25.788,-80.317,25.7949789,-80.28672341095405,0.7359028498396194,3.1289947807252796,large_airport,large_airport,0.0,-4.0,0.0,-15.0,null,null,null,null,null,1321,1603,15.0,9.0,1336,1554,162.0,138.0,0.0,null,0.0


Databricks data profile. Run in Databricks to view.

flight_id,prediction_utc,origin_obs_utc,asof_minutes,origin_station_id,dest_station_id,FL_DATE,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER,OP_CARRIER_FL_NUM,TAIL_NUM,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,ORIGIN,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_STATE_ABR,DISTANCE,DISTANCE_GROUP,HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyWetBulbTemperature,HourlyPrecipitation,HourlyWindSpeed,HourlyWindDirection,HourlyWindGustSpeed,HourlyVisibility,HourlyRelativeHumidity,HourlyStationPressure,HourlySeaLevelPressure,HourlyAltimeterSetting,HourlySkyConditions,HourlyPresentWeatherType,origin_station_lat,origin_station_lon,origin_airport_lat,origin_airport_lon,dest_station_lat,dest_station_lon,dest_airport_lat,dest_airport_lon,origin_station_dis,dest_station_dis,origin_type,dest_type,DEP_DEL15,DEP_DELAY,ARR_DEL15,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DEP_TIME,ARR_TIME,TAXI_OUT,TAXI_IN,WHEELS_OFF,WHEELS_ON,ACTUAL_ELAPSED_TIME,AIR_TIME,CANCELLED,CANCELLATION_CODE,DIVERTED
2019-12-31|YX|6140|ROC|LGA,2019-12-31T10:00:00Z,2019-12-31T09:54:00Z,6,72529014768,72503014732,2019-12-31,2019,4,12,31,2,YX,YX,6140,N244JQ,700,832,92.0,ROC,14576,"Rochester, NY",NY,LGA,12953,"New York, NY",NY,254.0,2,38,31,35,0.00,9,210,null,10.00,76,28.99,29.60,29.58,FEW:02 35 BKN:07 60 OVC:08 200,null,43.117,-77.677,43.12759065,-77.66522075259782,40.779,-73.88,40.7757145,-73.87336398511545,1.5168292888850612,0.6676019544429856,medium_airport,large_airport,0.0,-7.0,0.0,-28.0,null,null,null,null,null,653,804,11.0,7.0,704,757,71.0,53.0,0.0,null,0.0
2019-12-31|YX|6139|ORD|BOS,2019-12-31T15:57:00Z,2019-12-31T15:51:00Z,6,72530094846,72509014739,2019-12-31,2019,4,12,31,2,YX,YX,6139,N860RW,1157,1523,146.0,ORD,13930,"Chicago, IL",IL,BOS,10721,"Boston, MA",MA,867.0,4,26,19,24,0.00,17,260,23,10.00,75,29.05,29.80,29.77,OVC:08 23,null,41.995,-87.934,41.97795725,-87.90917584851792,42.361,-71.01,42.3629737,-71.01350667543224,2.7930256931701187,0.362182815141868,large_airport,large_airport,1.0,32.0,1.0,22.0,22.0,0.0,0.0,0.0,0.0,1229,1545,23.0,6.0,1252,1539,136.0,107.0,0.0,null,0.0
2019-12-31|YX|6139|BOS|ORD,2019-12-31T12:20:00Z,2019-12-31T11:54:00Z,26,72509014739,72530094846,2019-12-31,2019,4,12,31,2,YX,YX,6139,N240JQ,920,1127,187.0,BOS,10721,"Boston, MA",MA,ORD,13930,"Chicago, IL",IL,867.0,4,39,35,37,0.00,10,330,null,7.00,86,29.52,29.55,29.55,OVC:08 6,null,42.361,-71.01,42.3629737,-71.01350667543224,41.995,-87.934,41.97795725,-87.90917584851792,0.362182815141868,2.7930256931701187,large_airport,large_airport,0.0,-2.0,0.0,-3.0,null,null,null,null,null,918,1124,26.0,29.0,944,1055,186.0,131.0,0.0,null,0.0
2019-12-31|YX|6133|BOS|ORD,2019-12-31T20:45:00Z,2019-12-31T19:54:00Z,51,72509014739,72530094846,2019-12-31,2019,4,12,31,2,YX,YX,6133,N204JQ,1745,1956,191.0,BOS,10721,"Boston, MA",MA,ORD,13930,"Chicago, IL",IL,867.0,4,39,33,37,0.00,8,260,null,10.00,79,29.53,29.56,29.56,FEW:02 8 BKN:07 10 OVC:08 65,null,42.361,-71.01,42.3629737,-71.01350667543224,41.995,-87.934,41.97795725,-87.90917584851792,0.362182815141868,2.7930256931701187,large_airport,large_airport,0.0,-8.0,0.0,-39.0,null,null,null,null,null,1737,1917,16.0,10.0,1753,1907,160.0,134.0,0.0,null,0.0
2019-12-31|YX|6131|LGA|BUF,2019-12-31T20:00:00Z,2019-12-31T19:51:00Z,9,72503014732,72528014733,2019-12-31,2019,4,12,31,2,YX,YX,6131,N203JQ,1700,1837,97.0,LGA,12953,"New York, NY",NY,BUF,10792,"Buffalo, NY",NY,292.0,2,45,35,41,0.00,10,210,null,10.00,68,29.59,29.62,29.62,BKN:07 50 OVC:08 60,null,40.779,-73.88,40.7757145,-73.87336398511545,42.941,-78.736,42.9402016,-78.73075229745353,0.6676019544429856,0.4362986842122279,large_airport,large_airport,0.0,-12.0,0.0,-36.0,null,null,null,null,null,1648,1801,12.0,6.0,1700,1755,73.0,55.0,0.0,null,0.0


In [0]:
from pyspark.sql import functions as F

# Assuming 'YEAR' and 'MONTH' are the columns representing year and month in df_joined_5Y_2015
year_col = 'YEAR'
month_col = 'MONTH'
cols = [c for c in df_joined_5Y_2015.columns if c not in [year_col, month_col]]

exprs = [
    (F.sum(F.col(c).isNull().cast("int")) / F.count(F.lit(1))).alias(f"{c}_null_pct")
    for c in cols
]

result = df_joined_5Y_2015.groupBy(year_col, month_col).agg(*exprs)
display(result.orderBy(year_col, month_col))

YEAR,MONTH,flight_id_null_pct,prediction_utc_null_pct,origin_obs_utc_null_pct,asof_minutes_null_pct,origin_station_id_null_pct,dest_station_id_null_pct,FL_DATE_null_pct,QUARTER_null_pct,DAY_OF_MONTH_null_pct,DAY_OF_WEEK_null_pct,OP_UNIQUE_CARRIER_null_pct,OP_CARRIER_null_pct,OP_CARRIER_FL_NUM_null_pct,TAIL_NUM_null_pct,CRS_DEP_TIME_null_pct,CRS_ARR_TIME_null_pct,CRS_ELAPSED_TIME_null_pct,ORIGIN_null_pct,ORIGIN_AIRPORT_ID_null_pct,ORIGIN_CITY_NAME_null_pct,ORIGIN_STATE_ABR_null_pct,DEST_null_pct,DEST_AIRPORT_ID_null_pct,DEST_CITY_NAME_null_pct,DEST_STATE_ABR_null_pct,DISTANCE_null_pct,DISTANCE_GROUP_null_pct,HourlyDryBulbTemperature_null_pct,HourlyDewPointTemperature_null_pct,HourlyWetBulbTemperature_null_pct,HourlyPrecipitation_null_pct,HourlyWindSpeed_null_pct,HourlyWindDirection_null_pct,HourlyWindGustSpeed_null_pct,HourlyVisibility_null_pct,HourlyRelativeHumidity_null_pct,HourlyStationPressure_null_pct,HourlySeaLevelPressure_null_pct,HourlyAltimeterSetting_null_pct,HourlySkyConditions_null_pct,HourlyPresentWeatherType_null_pct,origin_station_lat_null_pct,origin_station_lon_null_pct,origin_airport_lat_null_pct,origin_airport_lon_null_pct,dest_station_lat_null_pct,dest_station_lon_null_pct,dest_airport_lat_null_pct,dest_airport_lon_null_pct,origin_station_dis_null_pct,dest_station_dis_null_pct,origin_type_null_pct,dest_type_null_pct,DEP_DEL15_null_pct,DEP_DELAY_null_pct,ARR_DEL15_null_pct,ARR_DELAY_null_pct,CARRIER_DELAY_null_pct,WEATHER_DELAY_null_pct,NAS_DELAY_null_pct,SECURITY_DELAY_null_pct,LATE_AIRCRAFT_DELAY_null_pct,DEP_TIME_null_pct,ARR_TIME_null_pct,TAXI_OUT_null_pct,TAXI_IN_null_pct,WHEELS_OFF_null_pct,WHEELS_ON_null_pct,ACTUAL_ELAPSED_TIME_null_pct,AIR_TIME_null_pct,CANCELLED_null_pct,CANCELLATION_CODE_null_pct,DIVERTED_null_pct
2015,1,0.0,5.489735471351241E-4,0.0013277499744663466,0.0013277499744663466,5.489735471351241E-4,5.489735471351241E-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005919551969495796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00174267184148708,0.0017831001259660231,0.010813502195894189,0.20491820719708576,0.001595853334695128,0.004583290777244408,0.8977802744016614,0.001468185067919518,0.001978858135021959,0.010245378408742723,0.08325886017771422,0.15741497293432744,0.08210984577673373,0.8414040955979981,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,5.489735471351241E-4,0.02480381643005481,0.02480381643005481,0.027565706601300515,0.027565706601300515,0.7958350355768904,0.7958350355768904,0.7958350355768904,0.7958350355768904,0.7958350355768904,0.02480381643005481,0.026110288360058558,0.025269805603785788,0.026110288360058558,0.025269805603785788,0.026110288360058558,0.027565706601300515,0.027565706601300515,0.0,0.9745046471249106,0.0
2015,2,0.0,5.475417704471902E-4,0.0012442012996544663,0.0012442012996544663,5.475417704471902E-4,5.452118054665639E-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00837389414037107,0.0,0.0,4.6599299612526824E-6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0016146657315740545,0.0017404838405278768,0.012945285432359952,0.19999021414708137,0.0017288340156247453,0.005694434412650778,0.8738556959488899,0.001544766782155264,0.0017567935953922613,0.012658699739742912,0.08040476151643441,0.15683926270588153,0.08306325155932906,0.8671337469797828,5.475417704471902E-4,5.475417704471902E-4,5.475417704471902E-4,5.475417704471902E-4,5.452118054665639E-4,5.452118054665639E-4,5.452118054665639E-4,5.452118054665639E-4,5.475417704471902E-4,5.452118054665639E-4,5.475417704471902E-4,5.452118054665639E-4,0.04673676754638378,0.04673676754638378,0.050159486102923874,0.050159486102923874,0.7782362631089654,0.7782362631089654,0.7782362631089654,0.7782362631089654,0.7782362631089654,0.04673676754638378,0.048514530826601676,0.04755924518454488,0.048514530826601676,0.04755924518454488,0.048514530826601676,0.050159486102923874,0.

In [0]:
df_ppg_flights = df_joined_5Y_2015.filter((df_joined_5Y_2015.ORIGIN == "PPG") | (df_joined_5Y_2015.DEST == "PPG"))
display(df_ppg_flights)

flight_id,prediction_utc,origin_obs_utc,asof_minutes,origin_station_id,dest_station_id,FL_DATE,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER,OP_CARRIER_FL_NUM,TAIL_NUM,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,ORIGIN,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_STATE_ABR,DISTANCE,DISTANCE_GROUP,HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyWetBulbTemperature,HourlyPrecipitation,HourlyWindSpeed,HourlyWindDirection,HourlyWindGustSpeed,HourlyVisibility,HourlyRelativeHumidity,HourlyStationPressure,HourlySeaLevelPressure,HourlyAltimeterSetting,HourlySkyConditions,HourlyPresentWeatherType,origin_station_lat,origin_station_lon,origin_airport_lat,origin_airport_lon,dest_station_lat,dest_station_lon,dest_airport_lat,dest_airport_lon,origin_station_dis,dest_station_dis,origin_type,dest_type,DEP_DEL15,DEP_DELAY,ARR_DEL15,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DEP_TIME,ARR_TIME,TAXI_OUT,TAXI_IN,WHEELS_OFF,WHEELS_ON,ACTUAL_ELAPSED_TIME,AIR_TIME,CANCELLED,CANCELLATION_CODE,DIVERTED
2017-10-13|HA|466|PPG|HNL,2017-10-14T08:30:00Z,2017-10-14T07:53:00Z,37,91197521510,91182022521,2017-10-13,2017,4,10,13,5,HA,HA,466,N590HA,2330,600,330.0,PPG,14222,"Pago Pago, TT",TT,HNL,12173,"Honolulu, HI",HI,2599.0,11,82,68,73,0.00,0,000,null,10.00,63,29.88,29.93,29.93,CLR:00,null,19.736,-156.049,-14.3347658,-170.71596862723766,21.324,-157.929,21.32040435,-157.9173135198164,4114.283979810308,1.2748484015654673,medium_airport,large_airport,1.0,51.0,1.0,29.0,0.0,0.0,0.0,0.0,29.0,21,629,11.0,7.0,32,622,308.0,290.0,0.0,null,0.0
2017-12-22|HA|466|PPG|HNL,2017-12-23T08:25:00Z,2017-12-23T07:53:00Z,32,91197521510,91182022521,2017-12-22,2017,4,12,22,5,HA,HA,466,N580HA,2325,550,325.0,PPG,14222,"Pago Pago, TT",TT,HNL,12173,"Honolulu, HI",HI,2599.0,11,72,48,59,0.00,16,070,null,10.00,43,29.89,29.93,29.94,CLR:00,null,19.736,-156.049,-14.3347658,-170.71596862723766,21.324,-157.929,21.32040435,-157.9173135198164,4114.283979810308,1.2748484015654673,medium_airport,large_airport,0.0,-16.0,0.0,-25.0,null,null,null,null,null,2309,525,6.0,6.0,2315,519,316.0,304.0,0.0,null,0.0
2018-04-02|HA|466|PPG|HNL,2018-04-03T08:25:00Z,2018-04-03T07:53:00Z,32,91197521510,91182022521,2018-04-02,2018,2,4,2,1,HA,HA,466,N388HA,2325,550,325.0,PPG,14222,"Pago Pago, TT",TT,HNL,12173,"Honolulu, HI",HI,2599.0,11,77,69,72,0.00,11,160,null,10.00,77,29.80,29.85,29.85,BKN:07 80,null,19.736,-156.049,-14.3347658,-170.71596862723766,21.324,-157.929,21.32040435,-157.9173135198164,4114.283979810308,1.2748484015654673,medium_airport,large_airport,0.0,14.0,1.0,22.0,13.0,0.0,0.0,0.0,9.0,2339,612,15.0,12.0,2354,600,333.0,306.0,0.0,null,0.0
2015-07-10|HA|466|PPG|HNL,2015-07-11T08:20:00Z,2015-07-11T07:53:00Z,27,91197521510,91182022521,2015-07-10,2015,3,7,10,5,HA,HA,466,N587HA,2320,540,320.0,PPG,14222,"Pago Pago, TT",TT,HNL,12173,"Honolulu, HI",HI,2599.0,11,78,74,75,0.00,0,000,null,10.00,87,29.91,29.96,29.96,SCT:04 50 OVC:08 200,null,19.736,-156.049,-14.3347658,-170.71596862723766,21.324,-157.929,21.32040435,-157.9173135198164,4114.283979810308,1.2748484015654673,medium_airport,large_airport,0.0,-7.0,0.0,-10.0,null,null,null,null,null,2313,530,6.0,7.0,2319,523,317.0,304.0,0.0,null,0.0
2017-12-08|HA|466|PPG|HNL,2017-12-09T08:25:00Z,2017-12-09T07:53:00Z,32,91197521510,91182022521,2017-12-08,2017,4,12,8,5,HA,HA,466,N581HA,2325,550,325.0,PPG,14222,"Pago Pago, TT",TT,HNL,12173,"Honolulu, HI",HI,2599.0,11,73,60,65,0.00,5,100,null,10.00,64,29.93,29.98,29.98,CLR:00,null,19.736,-156.049,-14.3347658,-170.71596862723766,21.324,-157.929,21.32040435,-157.9173135198164,4114.283979810308,1.2748484015654673,medium_airport,large_airport,0.0,1.0,0.0,5.0,null,null,null,null,null,2326,555,10.0,7.0,2336,548,329.0,312.0,0.0,null,0.0
2016-08-03|HA|466|PPG|HNL,2016-08-04T08:25:00Z,2016-08-04T07:53:00Z,32,91197521510,91182022521,2016-08-03,2016,3,8,3,3,HA,HA,466,N580HA,2325,550,325.0,PPG,14222,"Pago Pago, TT",TT,HNL